In [ ]:
!pip install -q transformers datasets

In [ ]:
!pip install -q evaluate

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
save_path = "/content/drive/MyDrive/beam_emotion_model/"
import os
os.makedirs(save_path, exist_ok=True)

In [3]:
from datasets import load_dataset

# Load GoEmotions dataset (we use "simplified" 27-label version)
dataset = load_dataset("go_emotions", "simplified")

# Check sample
print(dataset["train"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.40k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

{'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [27], 'id': 'eebbqej'}


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

num_labels = 27  # GoEmotions simplified version has 27 labels

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
NUM_LABELS = 27

def preprocess_function(examples):
    texts = examples["text"]
    # Pick first label
    raw_labels = [label[0] for label in examples["labels"]]
    # Clamp labels to [0, NUM_LABELS-1]
    safe_labels = [label if label < NUM_LABELS else 0 for label in raw_labels]
    encodings = tokenizer(texts, padding="max_length", truncation=True)
    encodings["labels"] = safe_labels
    return encodings

encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./beam_goemotions_roberta",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [7]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.430000,1.319818,0.597862
2,1.196000,1.321209,0.590122


Epoch,Training Loss,Validation Loss,Accuracy
1,1.430000,1.319818,0.597862
2,1.196000,1.321209,0.590122
3,1.058000,1.296549,0.599889


TrainOutput(global_step=4071, training_loss=1.2267241340923942, metrics={'train_runtime': 5063.8333, 'train_samples_per_second': 25.718, 'train_steps_per_second': 0.804, 'total_flos': 3.42726440191488e+16, 'train_loss': 1.2267241340923942, 'epoch': 3.0})

In [12]:
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/beam_emotion_model/tokenizer_config.json',
 '/content/drive/MyDrive/beam_emotion_model/special_tokens_map.json',
 '/content/drive/MyDrive/beam_emotion_model/vocab.json',
 '/content/drive/MyDrive/beam_emotion_model/merges.txt',
 '/content/drive/MyDrive/beam_emotion_model/added_tokens.json',
 '/content/drive/MyDrive/beam_emotion_model/tokenizer.json')